In [3]:
# -*- coding: utf-8 -*-
"""
在 Jupyter Notebook 中运行
从 3500 常用汉字 txt 文件中提取每个汉字的【最常见声调】
使用 lazy_pinyin，结果准确，不引入多音字干扰
"""

import json
import sys
import subprocess

# 尝试导入 pypinyin，未安装则自动安装
try:
    from pypinyin import lazy_pinyin, Style
    print("✅ pypinyin 已加载")
except ImportError:
    print("🔍 pypinyin 未安装，正在自动安装...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pypinyin"])
    from pypinyin import lazy_pinyin, Style
    print("✅ pypinyin 安装完成")

def extract_tone_for_chars(input_file_path, output_json_path):
    """
    读取txt中的汉字，获取每个字最常见的声调，保存为JSON
    """
    char_tone_map = {}

    try:
        # 读取文件
        with open(input_file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        # 提取汉字（Unicode 范围 \u4e00 - \u9fff）
        chinese_chars = [char for char in content if '\u4e00' <= char <= '\u9fff']
        
        # 去重并保持顺序
        seen = set()
        unique_chars = []
        for char in chinese_chars:
            if char not in seen:
                seen.add(char)
                unique_chars.append(char)
        
        print(f"📊 共读取到 {len(chinese_chars)} 个字符，其中不重复汉字 {len(unique_chars)} 个。")

        # 处理每个汉字
        for i, char in enumerate(unique_chars, 1):
            # 获取最常见拼音（如 'yuan2'）
            pinyin_list = lazy_pinyin(
                char,
                style=Style.TONE3,
                errors='ignore'  # 忽略无法识别的字符
            )
            
            tone = None
            if pinyin_list and len(pinyin_list) > 0:
                full_pinyin = pinyin_list[0]
                if full_pinyin and full_pinyin[-1].isdigit():
                    tone = int(full_pinyin[-1])  # 声调 1-4
                elif full_pinyin:  # 轻声，如 'ma'
                    tone = 0
            # 如果无法识别，tone 保持 None 或可设为 0
            
            char_tone_map[char] = tone if tone is not None else 0  # 未知字默认为 0（可选）
            
            # 每500个字打印一次进度
            if i % 500 == 0 or i == len(unique_chars):
                print(f"🔄 处理中: {i}/{len(unique_chars)}")

        # 保存为 JSON
        with open(output_json_path, 'w', encoding='utf-8') as json_file:
            json.dump(char_tone_map, json_file, ensure_ascii=False, indent=2)
        
        print(f"\n🎉 成功！所有汉字的声调已保存到：")
        print(f"   📄 {output_json_path}")
        print(f"\n🔍 示例数据格式：")
        print(f"   {{'一': 1, '丁': 1, '七': 1, '人': 2, '入': 4, '儿': 2, '员': 2}}")
        print(f"   📌 说明：1=第一声，2=第二声，3=第三声，4=第四声，0=轻声或未知")

    except FileNotFoundError:
        print(f"❌ 错误：找不到文件 '{input_file_path}'")
        print(f"   请检查文件名是否正确，是否与本 Notebook 在同一目录。")
    except Exception as e:
        print(f"❌ 发生错误：{e}")

# ==================== 用户配置区 ====================
# 🔧 请修改为您的实际文件名
input_txt = "常用汉字表.txt"      # 您的汉字txt文件
output_json = "char_tones.json"          # 输出的JSON文件名

# ==================== 运行函数 ====================
extract_tone_for_chars(input_txt, output_json)

✅ pypinyin 已加载
📊 共读取到 7827 个字符，其中不重复汉字 7826 个。
🔄 处理中: 500/7826
🔄 处理中: 1000/7826
🔄 处理中: 1500/7826
🔄 处理中: 2000/7826
🔄 处理中: 2500/7826
🔄 处理中: 3000/7826
🔄 处理中: 3500/7826
🔄 处理中: 4000/7826
🔄 处理中: 4500/7826
🔄 处理中: 5000/7826
🔄 处理中: 5500/7826
🔄 处理中: 6000/7826
🔄 处理中: 6500/7826
🔄 处理中: 7000/7826
🔄 处理中: 7500/7826
🔄 处理中: 7826/7826

🎉 成功！所有汉字的声调已保存到：
   📄 char_tones.json

🔍 示例数据格式：
   {'一': 1, '丁': 1, '七': 1, '人': 2, '入': 4, '儿': 2, '员': 2}
   📌 说明：1=第一声，2=第二声，3=第三声，4=第四声，0=轻声或未知
